# Building an Extensible Neural Network
---

Grabbing the necessary imports.

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
LOG_DIR = "/tmp/tensorflow/mnist/"
DAT_DIR = "/tmp/tensorflow/mnist/data/"
LRN_RAT = 0.1

mnist = input_data.read_data_sets(DAT_DIR, one_hot=True)
_epochs = 6
BATCHES = 100
NXAMPLS = (_epochs * mnist.train.images.shape[0]) // BATCHES

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/tensorflow/mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/tensorflow/mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/tensorflow/mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/tensorflow/mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
if tf.gfile.Exists(LOG_DIR):
    tf.gfile.DeleteRecursively(LOG_DIR)
tf.gfile.MakeDirs(LOG_DIR)

In [4]:
sess = tf.InteractiveSession()

In [5]:
with tf.name_scope("input"):
    x  = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y_ = tf.placeholder(tf.float32, [None,  10], name="y-input")

In [6]:
with tf.name_scope("input_reshape"):
    img_shape_inp = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image("input", img_shape_inp, 10)

In [7]:
def var_biases(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)

In [8]:
def var_weight(shape):
    init = tf.truncated_normal(shape, stddev=pow(shape[0], -0.5))
    return tf.Variable(init)

In [9]:
def var_summarize(var):
    with tf.name_scope("summaries"):
        
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev)
        
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        
        tf.summary.histogram("histogram", var)    

In [10]:
def Layer(name, inp_ten, inp_dim, out_dim, act=tf.nn.sigmoid):
    with tf.name_scope(name):
        
        with tf.name_scope("weights"):
            weights = var_weight([inp_dim, out_dim])
            var_summarize(weights)
            
#         with tf.name_scope("biases"):
#             biases  = var_biases([out_dim])
#             var_summarize(biases)
            
        with tf.name_scope("Wx_plus_b"):
            pre_activation = tf.matmul(inp_ten, weights)# + biases
            tf.summary.histogram("pre_activation", pre_activation)
            
        activation = act(pre_activation, name="activation")
        tf.summary.histogram("activations", activation)
        
        return activation

In [11]:
def arch_check(nodes):
    return len(nodes) >= 2

In [12]:
nodes = [784, 200, 200, 200, 10]

layers = [x]

if arch_check(nodes):
    for idx in range(len(nodes) - 2):
        name = "hidden_{}".format(idx)
        prev = layers[idx]
        inp_dim = nodes[idx]
        out_dim = nodes[idx + 1]
        new_lyr = Layer(name, prev, inp_dim, out_dim)
        layers.append(new_lyr)

    # Do not apply softmax activation yet, see below.
    y = Layer("output", layers[-1], nodes[-2], nodes[-1], act=tf.identity)

In [13]:
with tf.name_scope("cross_entropy"):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    
    with tf.name_scope("total"):
        xentropy = tf.reduce_mean(diff)
        
tf.summary.scalar("xentropy", xentropy)

<tf.Tensor 'xentropy:0' shape=() dtype=string>

https://github.com/tensorflow/tensorflow/blob/r1.4/tensorflow/python/training/gradient_descent.py

In [14]:
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(LRN_RAT).minimize(xentropy)

In [ ]:
with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
tf.summary.scalar("accuracy", accuracy)

In [ ]:
merged = tf.summary.merge_all()
trn_writer = tf.summary.FileWriter(LOG_DIR + "train/", sess.graph)
tst_writer = tf.summary.FileWriter(LOG_DIR + "test/")
tf.global_variables_initializer().run()

In [ ]:
def feed_dict(train):
    xs, ys = mnist.train.next_batch(BATCHES) if train else (mnist.test.images, mnist.test.labels)
    
    return {x: xs, y_: ys}

In [ ]:
for step in range(NXAMPLS):
    if step % 10 == 0:
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
        tst_writer.add_summary(summary, step)
#         print("Accuracy at step {}: {}".format(step, acc))
    else:
        if step % 100 == 99:
            run_opts = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_meta = tf.RunMetadata()
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True), 
                                  options=run_opts, run_metadata=run_meta)
            trn_writer.add_run_metadata(run_meta, "step{:03d}".format(step))
            trn_writer.add_summary(summary, step)
#             print("Adding run metadata for {}".format(step))
        else:
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
            trn_writer.add_summary(summary, step)
        
trn_writer.close()
tst_writer.close()